### Point csv to feature csv

In [1]:
from feature_process import *
import numpy as np
import os

In [2]:
# get csv paths
paths  = []
for root,dirs,files in os.walk('G:/mice/dataset/landmark/top_crop_mars/20210811/'):
    root = root.replace('\\','/')
    if(root.find('')!=-1): #select mice
        for f in files:
            if(f.find('.csv')!=-1): #select file 
                paths.append(root+'/'+f)

In [3]:
def get_name(path):
    sp = path.split('/')
    name = sp[-2]
    if sp[-1].find('basal')!=-1:
        bt = 'basal'
    else:
        bt = 'treat'
    return name+'-'+bt+'.csv'

In [4]:
# dlc_path = paths[1]
# dlc1 = dlc(dlc_path)
# feat = combine_feat(dlc1.raw)
# np.savetxt("./feat/Vtreat1.csv",feat,delimiter=",")
# np.savetxt("./feat/Vbasal1.csv",feat,delimiter=",")

save_root = './feat/20210811/'
if not os.path.isdir(save_root):
    os.makedirs(save_root)
for path in paths:
    dlc1 = dlc(path)
    feat = combine_feat(dlc1.raw)
    fname = get_name(path)
    np.savetxt(save_root+fname,feat,delimiter=",")

### cluster output

In [2]:
import scipy.io
import numpy as np
import cv2
import imageio

In [3]:
seg = scipy.io.loadmat('seg.mat')['seg'].T
label = scipy.io.loadmat('label.mat')['label'].T

In [4]:
def seg_gif(vid_path, seg, label,out_path='./result'):
    '''
    save video clips gif to out_path by classes
    '''
    cap = cv2.VideoCapture(vid_path)

    for i in range(len(label[0])):
        cpath = out_path+'/'+str(i)+'/'
        if not os.path.isdir(cpath):
            os.makedirs(cpath)

    vid_i=0
    imgs = []
    segi=1
    i=0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break
        # frame in clip
        if i<seg[segi]:
            imgs.append(frame)
        # split point reached
        if i == seg[segi]:
            cpath = out_path+'/'+str(np.argmax(label[segi-1]))+'/'
            imageio.mimsave(cpath+"/"+str(vid_i)+".gif",imgs)
            imgs = []
            imgs.append(frame)
            vid_i += 1
            segi += 1
        i+=1
    cap.release()

In [5]:
seg_gif('./F2.avi',seg,label)